# Atmosphere
> Load and scale the maps of the noise from the atmosphere
- toc: True

* We have 8 independent realizations
* They scale with the number of telescopes, i.e. it doesn't make any difference if there are 1 or 3 channels in the same telescope, they do not help beat the atmosphere
* They were simulated for 10 days at 100% efficiency at Chile, 46.29% (SAT) and 37.23% (LAT) at Pole

In [ ]:
# default_exp atmosphere

In [ ]:
# export

import healpy as hp
import numpy as np
from path import Path
import logging as log

from s4refsimtool.core import get_telescope, base_folder, simulations_observing_efficiency

In [ ]:
import h5py
import toml

In [ ]:
config = toml.load("s4_reference_design.toml")

In [ ]:
log.basicConfig(level=log.INFO)

In [ ]:
# exports


def get_telecope_years(config, site, channel):
    """Compute the number of telescope/years in the CMB-S4 configuration

    config_telescopes : dict
        CMB-S4 telescopes configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    """
    telescope_years = 0
    for tube, bands in config["telescopes"][get_telescope(channel)].items():
        if bands["site"].lower() == site.lower():
            has_band = bands.get(channel[:-1], 0) > 0
            telescope_years += has_band * bands.get(
                "years", config["experiment"]["total_experiment_length_years"]
            )
    return telescope_years

In [ ]:
s4 = h5py.File("cmbs4_tophat.h5", 'r')

In [ ]:
!tail -30 s4_reference_design.toml

## Compute telescope/years for the reference design

In [ ]:
for site in ["Pole", "Chile"]:
    for channel in s4.keys():
        telescope_years = get_telecope_years(config["telescopes"], site, channel)
        print(site, channel, telescope_years)
        telescope = get_telescope(channel)
        if site == "Chile":
            if telescope == "SAT":
                assert telescope_years == 0, "All SAT at Pole"
            elif channel.startswith("ULFL"):
                assert telescope_years == 0, "No ULFL in Chile"              
            else:
                assert telescope_years == 14, "2 LAT each band"
        if site == "Pole":
            if telescope == "SAT":
                assert telescope_years == 14, "2 SAT telescopes each band"
            else:
                assert telescope_years == 7, "1 LAT with all 4 bands" 
    print(30 * "=")

In [ ]:
# exports


def load_atmosphere(config, site, channel, realization=0):
    """Load foreground maps for a channel

    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    realization : int
        Choose one of the available 8 realizations

    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """

    telescope = get_telescope(channel)
    map_filename = (
        Path(base_folder)
        / f"{realization:08d}"
        / f"{site.lower()}_atmosphere_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits"
    )
    log.info(f"Reading {map_filename}")
    atmosphere_map = hp.read_map(
        map_filename, (0, 1, 2), dtype=np.float32, verbose=False
    )
    atmosphere_map[atmosphere_map == hp.UNSEEN] = np.nan
    # input map is 10 days at 100% efficiency
    atmosphere_map *= np.sqrt(10) / np.sqrt(
        365.25 * config["experiment"]["observing_efficiency"]
    )
    # pole sims have lower efficiency
    atmosphere_map *= np.sqrt(
        simulations_observing_efficiency[site.lower()].get(telescope, 1)
    )
    atmosphere_map /= np.sqrt(get_telecope_years(config["telescopes"], site, channel))
    return atmosphere_map

## Run on a channel and plot results

Available atmosphere maps

In [ ]:
filenames = !ls /global/cscratch1/sd/keskital/s4sim/reference_tool/out/00000000/*atmo*

In [ ]:
import os.path
for f in map(os.path.basename, filenames):
    print(f)

In [ ]:
channel = "LFS1"
site = "Pole"

In [ ]:
output_map = load_atmosphere(config, site, channel, realization=0)

In [ ]:
np.nanmax(output_map)

In [ ]:
assert np.nanmax(output_map) < 1e-3 and np.nanmax(output_map) > 1e-5, \
    "Amplitude check failed"

In [ ]:
assert np.nanmin(output_map) > -1e-4 and np.nanmin(output_map) < 1e-6, \
    "Amplitude check failed"

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(output_map[0], min=-1e-5, max=1e-5, unit="K", title="Atmosphere I")

There are no systematics in the half-wave plate or bandpass mismatch, so almost all the atmosphere signal is rejected in polarization.

In [ ]:
hp.mollview(output_map[1], min=-1e-9, max=1e-9, unit="K", title="Atmosphere Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-9, max=1e-9, unit="K", title="Atmosphere U")